In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from numpy.polynomial.laguerre import laggauss
from numpy.polynomial.legendre import leggauss

# Gaussian Quadrature Notebook
This notebook implements the Gaussian Quadrature method from *Numerical Analysis* in multiple dimensions.

Gaussian Quadrature (below) is valid if $P(x)$ is any polynomial of degree less than $2n$. It is
$$
\int_{-1}^{1} P(x)\,dx = \sum_{i=1}^{n} c_i P(x_i)
$$
Otherwise, it will just give a decently good approximation.

In [3]:
poly   = lambda x: 2*x**3 + 6*x**2 + 3*x + 6
expcos = lambda x: np.exp(x) * np.cos(x)
expsin = lambda x: x**6 - (x**2 * np.sin(2*x))

In [4]:
### Gaussian Quadrature Implementation
def gaussQuad1D(f, a, b, n):
    """1-D Gaussian Quadrature using Legendre Polynomials"""

    roots, weights = leggauss(n)

    sum = 0
    # perform the summation
    for i in range(n):
        # do transformation to account for all ranges
        # i have to do a negative here because the roots from leggaus
        # is negative of what I want
        x = linTransform(roots[i], a, b)
        sum += weights[i] * f(x)

    # multiply by the coefficents from the variable substitution
    return 0.5 * (b - a) * sum

def linTransform(t, a, b):
    """Linear transformation for turning bounds from (a, b) to (-1, 1) to be used in Gaussian quadrature"""
    return 0.5 * ((b - a) * t + a + b)

In [5]:
print(f"T1: {gaussQuad1D(poly, -1, 1, 3):0.7f}")
print(f"T2: {gaussQuad1D(expcos, -1, 1, 3):0.7f}")
print(f"T3: {gaussQuad1D(expsin, 1, 3, 3):0.7f}")

T1: 16.0000000
T2: 1.9333905
T3: 317.2641517


These match very well with the values provided in the book, so I'm confident in the routine.

## 2-D Gaussian Quadrature
The book provides the integral
$$
\int_{1.4}^{2.0} \int_{1.0}^{1.5} \ln (x+2y) \, dy \, dx \approx 0.4295545313.
$$

So, I'll base my initial construction on this thing. It looks to be defined as

$$
\int_{-1}^{1} \int_{-1}^{1} f(x, y) \, dy \, dx \approx \sum_{i=1}^{n} \sum_{j=1}^{n} c_i c_j P(x_i, x_j)
$$

In [6]:
def gaussQuad2D(f, x_bounds, y_bounds, n):
    # unpack variables needed for algo
    roots, weights = leggauss(n)
    a,b = x_bounds
    c,d = y_bounds

    # perform the double summation shown in the eq above
    sum = 0
    for i in range(n):
        # transform into x
        u = linTransform(roots[i], a, b)
        for j in range(n):
            # transform into y
            v = linTransform(roots[j], c, d)
            
            sum += weights[i] * weights[j] * f(u, v)

    # multiply by the extra coefficient needed from variable substitution
    return 0.25 * (b - a) * (d - c) * sum

In [7]:
ln_2d = lambda x, y: np.log(x + 2*y)
gaussian_2d = lambda x, y: np.exp(-x**2 -y**2)

In [8]:
print(gaussQuad2D(ln_2d, (1.4, 2.0), (1.0, 1.5), 3))
print(gaussQuad2D(gaussian_2d, (-1, 1), (-1, 1), 6))

0.4295545311524899
2.230983195257796


The first matches the value provided in the book, and the second agrees with the first few digits from WolframAlpha. Extrapolating this to 3-D is pretty trivial, so I will skip it.

## Gauss-Laguerre Quadrature
Figured I'd do this as a bonus- it does the following:
$$
\int_{0}^{\infty} e^{-x} f(x) \, dx \approx \sum_{i=1}^n c_i f(x_i)
$$

In [ ]:
def gaussLaguerre1D():